In [ ]:
%matplotlib inline
from astropy.table import Table
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np


mpl.rc('text', usetex=True)

In [ ]:
def poisson_upper(n, S):
    return n + S*np.sqrt(n + 0.75) + (S**2 + 3.)/4.
def poisson_lower(n, S):
    return n - S*np.sqrt(n - 0.25) + (S**2 - 1.)/4

def calc_1sigma_poisson_bounds(n, ntot):
    upper = poisson_upper(n, 1.0)
    lower = poisson_lower(n, 1.0)
    return [(n-lower)/ntot, (upper-n)/ntot]

# Load the classified catalog

In [ ]:
infall = Table().read('../catalogs/classified_surfacedensity.csv')
infall = infall[np.where((infall['morph'] != 'C') &
                         (infall['morph'] != 'U'))]
infall.sort('d_cl_mpc')

In [ ]:
# Sort into bins
infall['bin'] = -1*np.ones(len(infall), dtype='int8')
bins = np.arange(0, 5.001, 0.5)
nbins = len(bins)-1

for i in range(len(bins)-1):
    for row in infall:
        if bins[i] < row['d_cl_mpc'] < bins[i+1]:
            row['bin'] = i

# Plot the fractions in each bin

In [ ]:
def compute_binned_fractions(table, morph, nbins):
    binids = range(nbins)
    ntot, nmorph = np.ones(len(binids))*-1, np.ones(len(binids))*-1
    rhomin, rhomax = np.ones(len(binids))*-1, np.ones(len(binids))*-1
    rhomean = np.ones(len(binids))*-1
    for i in binids:
        onebin = table[np.where(table['bin'] == i)]
        ntot[i] = float(len(onebin))
        nmorph[i] = float(len(onebin[np.where(onebin['morph'] == morph)]))
        rhomin[i] = min(onebin['d_cl_mpc'])
        rhomax[i] = max(onebin['d_cl_mpc'])
        rhomean[i] = np.average(onebin['d_cl_mpc'])
    return nmorph, {'nmorph': nmorph, 'ntot': ntot, 'rhomin': rhomin, 'rhomax': rhomax, 'rhomean': rhomean}

#### Subplot details

In [ ]:
plotdeetz = [
    {
        'morph': ["E"], 
        'ylabel': r'$f_{\rm E}$',
        'oldcol': 'fE'
    },
    {
        'morph': ["S0"], 
        'ylabel': r'$f_{\rm S0}$',
        'oldcol': 'fS0'
    },
    {
        'morph': ["E", "S0"], 
        'ylabel': r'$f_{\rm E+S0}$',
        'oldcol': 'fearly'
    },
    {
        'morph': ["Irr", "Sa", "Sb", "Sc", "Sd", "Sm"], 
        'ylabel': r'$f_{\rm Sp+Irr}$',
        'oldcol': 'flate'
    }
]

#### Make the plot

In [ ]:
ebarkwargs = {'ms':2, 'elinewidth':0.5, 'capthick':0.5, 'capsize':2}

fig = plt.figure(figsize=(10, 5), dpi=250) 
gs = fig.add_gridspec(5,3) 

axs = [fig.add_subplot(gs[0,0]), fig.add_subplot(gs[0,1]), fig.add_subplot(gs[1:3,0]),
       fig.add_subplot(gs[1:3,1]), fig.add_subplot(gs[3:5,0]), fig.add_subplot(gs[3:5,1])]

xlabel = r'$r_{\rm cl}/R_{200}$'
yticks = (0.0, .2, .4, .6, .8, 1.)
xrange = (0, 5)

for ax in axs[:2]:
    ax.hist(infall['d_cl_mpc'], bins=bins)

for ax, opts in zip(axs[2:], plotdeetz):
    # Compute the fraction in infall region
    nmorphs = np.zeros(nbins)
    for morph in opts['morph']:
        nmorph_single, res = compute_binned_fractions(infall, morph, nbins)
        nmorphs+=nmorph_single
    
    # Plot the infall clusters by morphology
    ax.errorbar(res['rhomean'], nmorphs/res['ntot'], 
                xerr=[res['rhomean'] - res['rhomin'], res['rhomax'] - res['rhomean']],
                yerr=calc_1sigma_poisson_bounds(nmorphs, res['ntot']),
                fmt='ok', **ebarkwargs,)    
    
    # Format the axes
    ax.set_xlim(xrange)
    ax.set_ylim((-0.1, 1.1))
    ax.set_yticks(yticks)
    ax.set_ylabel(opts['ylabel'])
    ax.tick_params(which='both', direction='in', top=True, right=True, left=True, bottom=True)

# General axis formatting


axs[0].set_xticklabels(());

axs[1].set_xticklabels(());
axs[1].yaxis.tick_right()
axs[1].yaxis.set_label_position('right')

axs[2].set_xticklabels(());

axs[3].set_xticklabels(());
axs[3].yaxis.tick_right()
axs[3].yaxis.set_label_position('right')

axs[4].set_xlabel(xlabel)

axs[5].set_xlabel(xlabel)
axs[5].yaxis.tick_right()
axs[5].yaxis.set_label_position('right')


fig.subplots_adjust(wspace=.001, hspace=.001)
plt.savefig('plots/morphology-radius-relation.pdf', bbox_inches='tight')